create 11.04.2023 by Dirk Derichsweiler
# KAFKA - consumer

## create data

In [7]:
import random
import csv

######################
# Ezemeral Data Demo #
######################

# Define the list of products and stores
PRODUCTS = ['banana', 'apple', 'pear', 'grapes', 'orange', 'kiwi', 'watermelon', 'pomegranate', 'pineapple', 'mango', 'cucumber', 'carrot', 'capsicum', 'onion', 'potato', 'lemon', 'tomato', 'raddish', 'beetroot', 'cabbage', 'lettuce', 'spinach', 'soy bean', 'cauliflower', 'bell pepper', 'chilli pepper', 'turnip', 'corn', 'sweetcorn', 'sweet potato', 'paprika, jalepeño', 'ginger', 'garlic', 'peas', 'eggplant']
# create a list of fruits
FRUITS = ['banana', 'apple', 'pear', 'grapes', 'orange', 'kiwi', 'watermelon', 'pomegranate', 'pineapple', 'mango']
TYPES = ['fruits', 'vegetables']
COUNTRIES = ['Austria']
YEARS = [2018, 2019, 2020, 2021, 2022]
STORES = ['store1', 'store2', 'store3', 'store4', 'store5', 'store6', 'store7', 'store8', 'store9', 'store10']

# Define the number of rows to generate
num_rows = random.randint(1000, 5000)

data = []

# Generate the data
for _ in range(num_rows):
    product = random.choice(PRODUCTS)
    store = random.choice(STORES)
    year = random.choice(YEARS)
    unit_price = round(random.uniform(0.1, 10), 2)
    qty = round(random.uniform(100, 1000), 2)
    total_sales = round(unit_price * qty, 2)
    if product in FRUITS:
        data.append([
            product,
            'fruits',
            unit_price,
            'kg',
            qty,
            total_sales,
            store,
            random.choice(COUNTRIES),
            year
        ])
    else:
        data.append([
            product,
            'vegetables',
            unit_price,
            'kg',
            qty,
            total_sales,
            store,
            random.choice(COUNTRIES),
            year
        ])

        
        

## Kafka 

In [11]:
!pip install kafka-python

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 10.2 MB/s eta 0:00:00


In [21]:
from sshtunnel import open_tunnel
from time import sleep

with open_tunnel(
    ('16.103.6.85', 22),
    ssh_username="dfaasusr",
    ssh_pkey="../id_rsa_greenlake.pem",
    remote_bind_address=('127.0.0.1', 9092)
) as server:

    print(server.local_bind_port)
    while True:
        # press Ctrl-C for stopping
        sleep(1)

46501


KeyboardInterrupt: 

In [7]:
#!/bin/python3
### created by Dirk Derichsweiler+
#
# Demo for DF
import random
import csv
from kafka import KafkaProducer
import json
import sys
from sshtunnel import open_tunnel
from time import sleep

country = 'Austria'
currency = 'EUR'
kafka_server='localhost'
kafka_topic="sales_data"
store_amount = 1
start_year = 2021
end_year = 2023

# Define the products
products = [
        {"id": 1, "name": "banana", "type": "fruits", "unit_price": 0.5, "unit": "kg"},
        {"id": 2, "name": "apple", "type": "fruits", "unit_price": 0.7, "unit": "kg"},
        {"id": 3, "name": "pear", "type": "fruits", "unit_price": 0.6, "unit": "kg"},
        {"id": 4, "name": "grapes", "type": "fruits", "unit_price": 1.2, "unit": "kg"},
        {"id": 5, "name": "orange", "type": "fruits", "unit_price": 0.8, "unit": "kg"},
        {"id": 6, "name": "kiwi", "type": "fruits", "unit_price": 0.9, "unit": "piece"},
        {"id": 7, "name": "watermelon", "type": "fruits", "unit_price": 5.0, "unit": "kg"},
        {"id": 8, "name": "pomegranate", "type": "fruits", "unit_price": 2.5, "unit": "kg"},
        {"id": 9, "name": "pineapple", "type": "fruits", "unit_price": 1.5, "unit": "piece"},
        {"id": 10, "name": "mango", "type": "fruits", "unit_price": 2.0, "unit": "piece"},
        {"id": 11, "name": "cucumber", "type": "vegetables", "unit_price": 0.4, "unit": "piece"},
        {"id": 12, "name": "carrot", "type": "vegetables", "unit_price": 0.3, "unit": "kg"},
        {"id": 13, "name": "capsicum", "type": "vegetables", "unit_price": 0.5, "unit": "kg"},
        {"id": 14, "name": "onion", "type": "vegetables", "unit_price": 0.2, "unit": "kg"},
        {"id": 15, "name": "potato", "type": "vegetables", "unit_price": 0.3, "unit": "kg"},
        {"id": 16, "name": "lemon", "type": "vegetables", "unit_price": 0.4, "unit": "kg"},
        {"id": 17, "name": "tomato", "type": "vegetables", "unit_price": 0.6, "unit": "kg"},
        {"id": 18, "name": "raddish", "type": "vegetables", "unit_price": 0.3, "unit": "kg"},
        {"id": 19, "name": "beetroot", "type": "vegetables", "unit_price": 0.4, "unit": "kg"},
        {"id": 20, "name": "cabbage", "type": "vegetables", "unit_price": 0.5, "unit": "piece"},
        {"id": 21, "name": "lettuce", "type": "vegetables", "unit_price": 0.6, "unit": "piece"},
        {"id": 22, "name": "spinach", "type": "vegetables", "unit_price": 0.7, "unit": "kg"},
        {"id": 23, "name": "soy bean", "type": "vegetables", "unit_price": 0.8, "unit": "kg"},
        {"id": 24, "name": "cauliflower", "type": "vegetables", "unit_price": 0.6, "unit": "piece"},
        {"id": 25, "name": "bell pepper", "type": "vegetables", "unit_price": 0.7, "unit": "kg"},
        {"id": 26, "name": "chilli pepper", "type": "vegetables", "unit_price": 0.5, "unit": "kg"},
        {"id": 27, "name": "turnip", "type": "vegetables", "unit_price": 0.4, "unit": "kg"},
        {"id": 28, "name": "corn", "type": "vegetables", "unit_price": 0.3, "unit": "kg"},
        {"id": 29, "name": "sweetcorn", "type": "vegetables", "unit_price": 0.4, "unit": "kg"},
        {"id": 30, "name": "sweet potato", "type": "vegetables", "unit_price": 0.5, "unit": "kg"},
        {"id": 31, "name": "paprika", "type": "vegetables", "unit_price": 2.0, "unit": "kg"},
        {"id": 32, "name": "jalepeno", "type": "vegetables", "unit_price": 2.0, "unit": "kg"},
        {"id": 33, "name": "ginger", "type": "vegetables", "unit_price": 1.5, "unit": "kg"},
        {"id": 34, "name": "garlic", "type": "vegetables", "unit_price": 1.5, "unit": "kg"},
        {"id": 35, "name": "peas", "type": "vegetables", "unit_price": 3.0, "unit": "kg"},
        {"id": 36, "name": "eggplant", "type": "vegetables", "unit_price": 1.5, "unit": "kg"}
]

stores = []
years = []

if int(start_year) > int(end_year):
    print("E: start_year is higher then end year")
    sys.exit()

amount_of_years = int(end_year) - int(start_year)

x=0
while(x < int(amount_of_years+1)):
    years.append(str(int(start_year)+int(x)))
    x=x+1
print(years) 


# x = chr(ord('A') + 3)
x=0
while( x < int(store_amount)):

    char = chr(ord('A') + x )
    x=x+1
    stores.append("Store "+char)

# Sales data will be stored in a list of dictionaries
sales_data = []

for year in years:
    for store in stores:
        total_sales = 0

        # Generate sales data for vegetables
        for product in products:
            quantity = random.randint(50, 2000) # Generate a random quantity between 50 and 200 kg
            total_sales += quantity * product["unit_price"]

            # Add sales data to the list
            sales_data.append({
                "id": product["id"],
                "product": product["name"],
                "type": product["type"],
                "unit price": product["unit_price"],
                "unit": product["unit"],
                "qty": quantity,
                "sales": round(total_sales, 2),
                "currency": currency,
                "store": store,
                "country": country,
                "year": year
            })

with open_tunnel(
    ('16.103.6.85', 22),
    ssh_username="dfaasusr",
    ssh_pkey="../id_rsa_greenlake.pem",
    remote_bind_address=('16.103.6.85', 9092)
) as server:

    print("Kafka-Port: " + str(server.local_bind_port))
            
    # Define the Kafka producer configuration
    producer = KafkaProducer(
    bootstrap_servers=[kafka_server+str(server.local_bind_port)],
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

    producer.send(kafka_topic, value=sales_data)
    producer.send(kafka_topic, "EOF")
    producer.flush()

['2021', '2022', '2023']
Kafka-Port: 44223


NoBrokersAvailable: NoBrokersAvailable

In [2]:
!pip install sshtunnel

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 23.6 MB/s eta 0:00:0000:01


In [20]:
from sshtunnel import SSHTunnelForwarder
from kafka import KafkaProducer
import csv

server = SSHTunnelForwarder(
    '16.103.6.85',
    ssh_username="dfaasusr",
    ssh_pkey="../id_rsa_greenlake.pem",
    remote_bind_address=('localhost', 9092),
)

server.start()

print(server.local_bind_port)  # show assigned local port
# work with `SECRET SERVICE` through `server.local_bind_port`.

def copy_csv_file(input_file, output_topic, kafka_servers, completion_keyword):
    producer = KafkaProducer(bootstrap_servers=kafka_servers)

    with open(input_file, 'r') as file:
        csv_reader = csv.reader(file)

        ## Skip header if present
        #header = next(csv_reader, None)

        for row in csv_reader:
            # Convert row to string
            message = ','.join(row).encode('utf-8')
            # Publish the row as a message to the Kafka topic
            producer.send(output_topic, value=message)

        # Publish completion keyword
        producer.send(output_topic, value=completion_keyword.encode('utf-8'))

    producer.flush()
    producer.close()

# Configuration
input_file = '../demo_folder/data/Germany_sales_data_2019_2023.csv'
output_topic = 'aspire-test'
kafka_servers = 'localhost:'+str(server.local_bind_port)
completion_keyword = 'endofdemo'

# Copy CSV file to Kafka topic
copy_csv_file(input_file, output_topic, kafka_servers, completion_keyword)



server.stop()

41501


In [12]:
import paramiko
import sshtunnel

with sshtunnel.open_tunnel(
    ('16.103.6.85', 22),
    ssh_username="dfaasusr",
    ssh_pkey="/var/ssh/rsa_key",
    ssh_private_key_password="../id_rsa_greenlake.pem",
    remote_bind_address=('16.103.6.85', 22),
    local_bind_address=('0.0.0.0', 10022)
) as tunnel:
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect('127.0.0.1', 10022)
    # do some operations with client session
    client.close()

print('FINISH!')

2023-05-30 13:39:47,388| ERROR   | Could not open connection to gateway


BaseSSHTunnelForwarderError: Could not establish session to SSH gateway

In [10]:
from sshtunnel import SSHTunnelForwarder

# SSH connection details
ssh_host = '16.103.6.85'
ssh_username = 'dfaasusr'
ssh_key_path = '/home/dderichswei/notebooks/end2end/id_rsa_greenlake.pem'

# Remote service ports to forward
remote_ports = {
    9092: 9092,
    2049: 2049,
    111: 111
}

# Start the SSH tunnel
with SSHTunnelForwarder(
    (ssh_host, 22),
    ssh_username=ssh_username,
    ssh_private_key=ssh_key_path,
    local_bind_addresses=('0.0.0.0',22),
    remote_bind_addresses=list(remote_ports.values())
) as tunnel:
    # Print the local ports bound to the remote services
    for local_port, remote_port in remote_ports.items():
        print(f'Tunneling remote port {remote_port} to localhost:{local_port}')

    # Keep the script running until interrupted
    while True:
        pass


AssertionError: 